In [ ]:
# PART I

f = open('input10.txt', 'r')

map = []
starting_coordinates = (0,0)
pipe_options = {'|', '-', 'L', 'J', '7', 'F'}
pipe_directions = {
    '|': {'S': 'S', 'N': 'N'},
    '-': {'E': 'E', 'W': 'W'},
    'L': {'S': 'E', 'W': 'N'},
    'J': {'S': 'W', 'E': 'N'},
    '7': {'N': 'W', 'E': 'S'},
    'F': {'N': 'E', 'W': 'S'}
}
direction_options = {
  'N': {'|', 'L', 'J'},
  'S': {'|', '7', 'F'},
  'E': {'-', 'L', 'F'},
  'W': {'-', 'J', '7'}
}
corners = {'L', 'J', '7', 'F'}

# Put input into a 2D grid
for i, line in enumerate(f):
  pipes = []
  for j, tile in enumerate(line):
    pipes.append(tile)
    if tile == 'S':
      starting_coordinates = (i, j)
  map.append(pipes)

print(starting_coordinates)
print(map[starting_coordinates[0]][starting_coordinates[1]])
print('\n\n')

# Define helper methods

def search_for_direction(row, col):
  all_options = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
  for direction in all_options:
    coord = get_next_coordinates(direction, row, col)
    try:
      tile = map[coord[0]][coord[1]]
    except:
      pass
    if tile in pipe_options:
      result = get_direction(direction, tile)
      if result != 'You cannot enter pipe from this direction':
        return direction
  return 'You can take no valid direction from this tile'


def get_tile_type(row, col):
  all_options = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
  neighbors = []
  for direction in all_options:
    coord = get_next_coordinates(direction, row, col)
    try:
      tile = map[coord[0]][coord[1]]
    except:
      pass
    if tile in pipe_options:
      result = get_direction(direction, tile)
      if result != 'You cannot enter pipe from this direction':
        neighbors.append(direction)
  if len(neighbors) != 2:
    raise Exception('not proper number of movements from this tile')
  options1 = direction_options[neighbors[0]]
  options2 = direction_options[neighbors[1]]
  result = options1.intersection(options2)
  return list(result)[0]


def get_direction(previous_direction, pipe_type):
  if pipe_type not in pipe_options:
    raise Exception(f'Not a valid pipe type passed into get_direction: {pipe_type}')
  if previous_direction not in pipe_directions[pipe_type]:
    return 'You cannot enter pipe from this direction'
  return pipe_directions[pipe_type][previous_direction]


def get_next_coordinates(direction, row, col):
  match direction:
    case 'N': return (row-1, col)
    case 'NE': return (row-1, col+1)
    case 'E': return (row, col+1)
    case 'SE': return (row+1, col+1)
    case 'S': return (row+1, col)
    case 'SW': return (row+1, col-1)
    case 'W': return (row, col-1)
    case 'NW': return (row-1, col-1)


row, col = starting_coordinates[0], starting_coordinates[1]
direction = search_for_direction(row, col)
count = 0
while True:
  (row, col) = get_next_coordinates(direction, row, col)
  previous_direction = direction
  count += 1
  current_tile = map[row][col]
  if current_tile == 'S':
    break
  direction = get_direction(previous_direction, current_tile)


if (count % 2) != 0:
  count += 1

print(count/2)


(42, 8)
S



6838.0


In [ ]:
# PART 2

# Same as part 1, but keep track of the coordinates

f = open('input10.txt', 'r')

map = []
starting_coordinates = (0,0)
full_boundary_set = set()
corner_set = set()
boundary_list = []

# Put input into a 2D grid
for i, line in enumerate(f):
  pipes = []
  for j, tile in enumerate(line):
    pipes.append(tile)
    if tile == 'S':
      starting_coordinates = (i, j)
  map.append(pipes)


extreme_values = {
  "rowmin": len(map), "rowmax": 0,
  "colmin": len(map[0]), "colmax": 0
}
def update_extreme_values(row, col):
  if row > extreme_values['rowmax']:
    extreme_values['rowmax'] = row
  if row < extreme_values['rowmin']:
    extreme_values['rowmin'] = row
  if col > extreme_values['colmax']:
    extreme_values['colmax'] = col
  if col < extreme_values['colmin']:
    extreme_values['colmin'] = col


row, col = starting_coordinates[0], starting_coordinates[1]
direction = search_for_direction(row, col)
count = 0
tile_type = get_tile_type(row, col)
print(f'Starting tile type is {tile_type}')
full_boundary_set.add((row, col))
# if tile_type in corners:
#   corner_set.add((row, col))
boundary_list.append((row, col))
update_extreme_values(row, col)
while True:
  (row, col) = get_next_coordinates(direction, row, col)
  previous_direction = direction
  count += 1
  current_tile = map[row][col]
  if current_tile == 'S':
    break
  full_boundary_set.add((row, col))
  # if current_tile in corners:
  #   corner_set.add((row, col))
  boundary_list.append((row, col))
  update_extreme_values(row, col)
  direction = get_direction(previous_direction, current_tile)


tile_type = get_tile_type(starting_coordinates[0], starting_coordinates[1])
map[starting_coordinates[0]][starting_coordinates[1]] = tile_type


if (count % 2) != 0:
  count += 1
print(count/2)
print('\n')

def add_cast_passthroughs(previous_corner, curr_corner):
  if previous_corner == 'L' and curr_corner == '7':
    return 1
  elif previous_corner == 'L' and curr_corner == 'J':
    return 2
  elif previous_corner == 'F' and curr_corner == 'J':
    return 1
  elif previous_corner == 'F' and curr_corner == '7':
    return 2
  raise Exception('failed to have proper corners')


points_to_test = []
internal_points = []

for row in range(extreme_values['rowmin'], extreme_values['rowmax']+1):
  for col in range(extreme_values['colmin'], extreme_values['colmax']+1):
    points_to_test.append((row, col))

for point in points_to_test:
  if point in full_boundary_set:
    continue
  # if point[0] != 4:
  #   continue
  #print(f'Testing point {point}')
  curr = (point[0], 0)
  raycast_count = 0
  corner_flag = False
  previous_corner = None
  while curr != point:
    if curr in full_boundary_set:
      value = map[curr[0]][curr[1]]
      # print(f'{curr}: {value}')
      if value == '-':
        if not corner_flag:
          raise Exception('horizontal piece with supposedly no corner?')
        pass
      elif value == '|':
        if corner_flag:
          raise Exception('vertical piece after a supposed corner')
        raycast_count += 1
      else:
        if not corner_flag:
          if value in ('J', '7'):
            raise Exception('Entered corner from the west without a previous corner')
          corner_flag = True
          previous_corner = value
        else:
          raycast_count += add_cast_passthroughs(previous_corner, value)
          corner_flag = False
    curr = (curr[0], curr[1] + 1)
  if (raycast_count % 2) != 0:
    print(f'{point}: passed through {raycast_count} walls. Internal')
    internal_points.append(point)
  else:
    pass
    # print(f'{point}: passed through {raycast_count} walls. Not internal')

print(len(internal_points))
print(internal_points)


# from shapely.geometry import Polygon
# polygon = Polygon(borders)
# area = polygon.area
# print('\n\n')
# print(area)

# from matplotlib.path import Path
# import numpy as np

# x, y = np.meshgrid(np.arange(len(map)), np.arange(len(map[0]))) # make a canvas with coordinates
# x, y = x.flatten(), y.flatten()
# points = np.vstack((x,y)).T

# p = Path(boundary_list) # make a polygon

# print(len(points_to_test))
# print(len(boundary_list))
# grid = p.contains_points(points_to_test)
# print(grid)

# count = 0
# for item in grid:
#   if item:
#     count += 1
# print(count)
# count -= len(full_boundary_set)
# print(count)



Starting tile type is F
6838.0


(36, 88): passed through 59 walls. Internal
(36, 104): passed through 73 walls. Internal
(37, 96): passed through 63 walls. Internal
(38, 56): passed through 29 walls. Internal
(38, 87): passed through 43 walls. Internal
(39, 59): passed through 29 walls. Internal
(39, 104): passed through 59 walls. Internal
(40, 39): passed through 17 walls. Internal
(40, 73): passed through 39 walls. Internal
(40, 76): passed through 41 walls. Internal
(40, 94): passed through 55 walls. Internal
(41, 62): passed through 33 walls. Internal
(41, 94): passed through 51 walls. Internal
(42, 38): passed through 23 walls. Internal
(42, 84): passed through 51 walls. Internal
(42, 89): passed through 55 walls. Internal
(43, 66): passed through 45 walls. Internal
(43, 87): passed through 57 walls. Internal
(44, 41): passed through 21 walls. Internal
(44, 102): passed through 61 walls. Internal
(45, 46): passed through 25 walls. Internal
(45, 80): passed through 45 walls. Inter